In [2]:
#import libraries
import pandas as pd

In [3]:
#read dataset
data = pd.read_csv("kpopdataset.csv",sep="\t",delimiter = ";")
data.head()

,rank,name,agency,real name,group,age,height,weight,blood type,origin,country,position,industry,gender,curr date,birth date
0,1.0,Jinah,주식회사 씨그널엔터테인먼트그룹,Seo Jin Ah,Berry Chu,37.0,NaN,NaN,NaN,Seoul,South Korea,"Leader, Vocalist",K-pop,Female,23.04.2020,29.04.1983
1,2.0,Hayul,주식회사 씨그널엔터테인먼트그룹,Kim Ha Yul,Berry Chu,33.0,NaN,NaN,NaN,Seoul,South Korea,Vocalist,K-pop,Female,23.04.2020,18.10.1986
2,3.0,Jieun,주식회사 씨그널엔터테인먼트그룹,Han Ji Eun,Berry Chu,33.0,NaN,NaN,NaN,Seoul,South Korea,Vocalist,K-pop,Female,23.04.2020,12.01.1987
3,4.0,Yunmi,주식회사 씨그널엔터테인먼트그룹,Heo Yun Mi,Berry Chu,32.0,NaN,NaN,NaN,Incheon,South Korea,Vocalist,K-pop,Female,23.04.2020,23.03.1988
4,5.0,Haum,주식회사 씨그널엔터테인먼트그룹,Kim Haum,Berry Chu,29.0,NaN,NaN,NaN,Seoul,South Korea,"Vocalist, Maknae",K-pop,Female,23.04.2020,20.10.1990


In [4]:
data.shape

(1901, 16)

In [5]:
#missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1901 entries, 0 to 1900
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rank        1877 non-null   float64
 1   name        1877 non-null   object 
 2   agency      1877 non-null   object 
 3   real name   1548 non-null   object 
 4   group       1877 non-null   object 
 5   age         1705 non-null   float64
 6   height      1583 non-null   float64
 7   weight      1236 non-null   float64
 8   blood type  1332 non-null   object 
 9   origin      682 non-null    object 
 10  country     1871 non-null   object 
 11  position    1814 non-null   object 
 12  industry    1877 non-null   object 
 13  gender      1877 non-null   object 
 14  curr date   1877 non-null   object 
 15  birth date  1705 non-null   object 
dtypes: float64(4), object(12)
memory usage: 237.8+ KB


In [6]:
#drop the columns
data.drop('real name',axis=1,inplace=True)
data.drop('origin',axis=1,inplace=True)
data.drop('industry',axis=1,inplace=True)
data.drop('curr date',axis=1,inplace=True)
data.drop('birth date',axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1901 entries, 0 to 1900
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rank        1877 non-null   float64
 1   name        1877 non-null   object 
 2   agency      1877 non-null   object 
 3   group       1877 non-null   object 
 4   age         1705 non-null   float64
 5   height      1583 non-null   float64
 6   weight      1236 non-null   float64
 7   blood type  1332 non-null   object 
 8   country     1871 non-null   object 
 9   position    1814 non-null   object 
 10  gender      1877 non-null   object 
dtypes: float64(4), object(7)
memory usage: 163.5+ KB


In [10]:
popular = pd.read_csv("bestkpopgroups.csv",sep="\t",delimiter = ";")
popular.head()

,Rank,Group,Label
0,1,BTS (Bangtan Boys),Big Hit Entertainment
1,2,Black Pink,YG Entertaiment
2,3,GOT7,JYP Entertainment
3,4,EXO,SM Entertainment
4,5,Stray Kinds,JYP Entertainment
